In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from unidecode import unidecode

In [2]:
df_rawPadronData = pd.read_csv('../populationData/01_input/PadronMunicipal_202507.csv', sep=';')
df_padronData = df_rawPadronData[['DESC_DISTRITO', 'DESC_BARRIO', 'ESPANOLESHOMBRES', 'ESPANOLESMUJERES', 'EXTRANJEROSHOMBRES', 'EXTRANJEROSMUJERES']]
df_padronData = df_padronData.assign(total = df_padronData['ESPANOLESHOMBRES'] + df_padronData['ESPANOLESMUJERES'] + df_padronData['EXTRANJEROSHOMBRES'] + df_padronData['EXTRANJEROSMUJERES'])
df_padronData = df_padronData.groupby(['DESC_DISTRITO', 'DESC_BARRIO']).agg("sum").reset_index(drop=False)
df_padronData = df_padronData.rename({'DESC_DISTRITO' : 'DISTRITO', 'DESC_BARRIO' : 'BARRIO'}, axis=1)
df_padronData['DISTRITO'] = df_padronData['DISTRITO'].apply(lambda x: x.strip().lower()).map(unidecode)
df_padronData['BARRIO'] = df_padronData['BARRIO'].apply(lambda x: x.strip().lower()).map(unidecode)

df_padronData

,DISTRITO,BARRIO,ESPANOLESHOMBRES,ESPANOLESMUJERES,EXTRANJEROSHOMBRES,EXTRANJEROSMUJERES,total
0,arganzuela,acacias,15102,17703,1684,1832,36321
1,arganzuela,atocha,1021,1058,253,266,2598
2,arganzuela,chopera,7675,9413,1631,1733,20452
3,arganzuela,delicias,11848,13317,2063,2270,29498
4,arganzuela,imperial,9931,11365,1332,1440,24068
...,...,...,...,...,...,...,...
126,villaverde,angeles,13501,15598,4366,4742,38207
127,villaverde,butarque,9131,9772,2187,2311,23401
128,villaverde,los rosales,14094,15722,5071,5067,39954
129,villaverde,san cristobal,5418,5898,4793,3928,20037


In [3]:
gdf_distritosBarrios = gpd.read_file('./barrios/BARRIOS.shp')[['NOMDIS', 'BARRIO_MAY', 'geometry']].rename({'NOMDIS' : 'DISTRITO', 'BARRIO_MAY' : 'BARRIO'}, axis=1)
gdf_distritosBarrios['DISTRITO'] = gdf_distritosBarrios['DISTRITO'].apply(lambda x : x.strip().lower()).map(unidecode)
gdf_distritosBarrios['BARRIO'] = gdf_distritosBarrios['BARRIO'].apply(lambda x : x.strip().lower()).map(unidecode)

gdf_distritosBarrios.sort_values(by=['DISTRITO', 'BARRIO'])

,DISTRITO,BARRIO,geometry
7,arganzuela,acacias,"POLYGON ((440272.919 4472969.567, 440350.318 4..."
12,arganzuela,atocha,"POLYGON ((441325.874 4473288.949, 441335.184 4..."
8,arganzuela,chopera,"POLYGON ((441005.241 4471403.949, 440750.571 4..."
10,arganzuela,delicias,"POLYGON ((440924.144 4472490.011, 441109.833 4..."
6,arganzuela,imperial,"POLYGON ((439379.682 4472498.334, 439333.423 4..."
...,...,...,...
110,villaverde,angeles,"POLYGON ((441076.001 4468448.76, 441105.538 44..."
108,villaverde,butarque,"POLYGON ((444144.707 4464513.034, 444144.856 4..."
109,villaverde,los rosales,"POLYGON ((441941.844 4466974.287, 441930.867 4..."
107,villaverde,san cristobal,"POLYGON ((441224.682 4464925.894, 441171.808 4..."


In [4]:
df_padronData = df_padronData.replace(
    {'casco h.barajas': 'casco historico de barajas',
     'casco h.vicalvaro': 'casco historico de vicalvaro',
     'casco h.vallecas': 'casco historico de vallecas',
     'villaverde alto c.h.' : 'villaverde alto - casco historico de villaverde',
     'fuencarral-el pardo': 'fuencarral - el pardo',
     'pena grande' : 'penagrande',
     'moncloa-aravaca' : 'moncloa - aravaca',
     'san blas-canillejas' : "san blas - canillejas"
     }
    )
gdf_distritosBarrios = gdf_distritosBarrios.replace(
    {
        'pilar' : 'el pilar',
        'aguilas' : 'las aguilas',
        'palomenas  bajas' : 'palomeras bajas'
    }
)
gdf_PadronDataByBarrio = pd.merge(left=gdf_distritosBarrios, right=df_padronData, on=['DISTRITO', 'BARRIO'], how='left')
gdf_PadronDataByBarrio.set_crs(gdf_distritosBarrios.crs)


,DISTRITO,BARRIO,geometry,ESPANOLESHOMBRES,ESPANOLESMUJERES,EXTRANJEROSHOMBRES,EXTRANJEROSMUJERES,total
0,centro,palacio,"POLYGON ((440112.785 4474645.921, 440078.075 4...",8861,9288,2633,2815,23597
1,centro,embajadores,"POLYGON ((440277.382 4473980.839, 440368.002 4...",16184,15752,9626,6878,48440
2,centro,cortes,"POLYGON ((440780.52 4474528.375, 440907.759 44...",3805,3777,1600,1652,10834
3,centro,justicia,"POLYGON ((440991.949 4474492.423, 440907.759 4...",6569,6327,2654,2703,18253
4,centro,universidad,"POLYGON ((440517.952 4474758.368, 440476.812 4...",11969,12018,4578,4938,33503
...,...,...,...,...,...,...,...,...
126,barajas,alameda de osuna,"POLYGON ((450614.387 4479399.407, 450752.655 4...",8802,9661,723,897,20083
127,barajas,aeropuerto,"POLYGON ((451940.168 4484759.459, 452134.178 4...",724,706,235,263,1928
128,barajas,casco historico de barajas,"POLYGON ((451288.265 4480266.282, 451233.112 4...",3041,3273,961,1110,8385
129,barajas,timon,"POLYGON ((450698.531 4479953.328, 450639.81 44...",5263,5859,620,676,12418
